In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib
import os
from IPython.display import display
matplotlib.use('Agg')
%matplotlib inline
import pandas as pd
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from gather_metrics import *
def get_ranked_by_means_df(gather_df, metric_list):
    means_df = gather_df.xs(['test_metrics', 'Folds Mean'], level=['set','fold'])
    ranked_df = pd.DataFrame(data=np.zeros(shape=means_df.shape), columns=metric_list, dtype=str)
    for m, i in zip(metric_list, range(len(metric_list))):
        metric_df = means_df[m]
        sorted_df = metric_df.sort_values(ascending=False)
        mscore_list = sorted_df.tolist()
        models_ranked = sorted_df.index.tolist()
        m_order_mscore_list = [str(m) + ", " + str(round(r,4)) for m, r in zip(models_ranked, mscore_list)]
        ranked_df.iloc[:,i] = m_order_mscore_list
    return ranked_df

In [2]:
# load stage CV
model_directory = '../../../job_results_small/'
class_dirs = [model_directory+'/random_forest/stage_1/',
              model_directory+'/irv/stage_1/',
              model_directory+'/neural_networks/stage_1/',
              model_directory+'/docking/stage_1/']
k = 5
gather_df_2 = gather_dir_metrics(class_dirs[0], k)     
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[1], k))   
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[2], 20))
gather_df_2 = gather_df_2.append(gather_dir_metrics(class_dirs[3], k))

col_indices = list(range(10)) + list(range(15, 20)) + list(range(25, 105)) + list(range(145, 149)) + list(range(150, 183))
col_indices = [i for i in col_indices if 'Mean' not in gather_df_2.columns[i] and 'Median' not in gather_df_2.columns[i]]
col_indices = [i for i in col_indices if 'RMI-FANCM FP' in gather_df_2.columns[i]]
metric_names = list(gather_df_2.columns.values[col_indices])

gather_df_2 = gather_df_2[metric_names]
gather_df_2 = gather_df_2.dropna(axis=0)

gather_df_cv = gather_df_2

#setup comparison matrix and overlaps
dtk_dict = dtk_multi_metrics(gather_df_cv, list(range(len(col_indices))))
dtk_analysis_dict = analyze_dtk_dict(dtk_dict)
mm_comp_dicts = get_mean_median_comps(gather_df_cv, list(range(len(col_indices))))
agg_comp_dict = get_agg_comp(mm_comp_dicts, dtk_analysis_dict, mean_w=1, median_w=0, tukey_w=1)

---
## Compute Best Models for Each Metric Based on Mean VS DTK+Mean

In [3]:
simple_ranked_cv = get_ranked_by_means_df(gather_df_cv, metric_names)
complex_ranked_cv = get_model_ordering(agg_comp_dict, metric_names)

top_n = 1
req_metrics = ['ROC AUC RMI-FANCM FP', 'BEDROC AUC RMI-FANCM FP','PR auc.integral RMI-FANCM FP', 
               'NEF AUC RMI-FANCM FP', 'NEF_1 % RMI-FANCM FP', 'EF_1 % RMI-FANCM FP']

simple_top_n = simple_ranked_cv[req_metrics].iloc[:top_n,:]
complex_top_n = complex_ranked_cv[req_metrics].iloc[:top_n,:]

### Ranking by Folds Mean Only

In [4]:
display('Ranking by Means. Shows model-mean pairs.', simple_ranked_cv[req_metrics])

'Ranking by Means. Shows model-mean pairs.'

,ROC AUC RMI-FANCM FP,BEDROC AUC RMI-FANCM FP,PR auc.integral RMI-FANCM FP,NEF AUC RMI-FANCM FP,NEF_1 % RMI-FANCM FP,EF_1 % RMI-FANCM FP
0,"RandomForest_h, 0.8346","RandomForest_h, 0.5114","RandomForest_h, 0.1237","RandomForest_h, 0.5439","RandomForest_h, 0.2782","RandomForest_h, 27.8179"
1,"RandomForest_g, 0.8306","RandomForest_g, 0.5045","RandomForest_g, 0.1225","RandomForest_g, 0.5324","RandomForest_g, 0.2738","RandomForest_g, 27.3831"
2,"RandomForest_e, 0.8256","RandomForest_e, 0.5001","RandomForest_e, 0.1224","RandomForest_e, 0.527","RandomForest_d, 0.2608","RandomForest_d, 26.0783"
3,"RandomForest_d, 0.8225","RandomForest_d, 0.4967","RandomForest_b, 0.1219","RandomForest_b, 0.5241","RandomForest_e, 0.2563","RandomForest_e, 25.6339"
4,"RandomForest_f, 0.8207","RandomForest_b, 0.4908","RandomForest_a, 0.1218","RandomForest_d, 0.5237","RandomForest_a, 0.2474","RandomForest_a, 24.7357"
5,"RandomForest_c, 0.8161","RandomForest_a, 0.4898","RandomForest_d, 0.1213","RandomForest_a, 0.5234","RandomForest_b, 0.2474","RandomForest_b, 24.7357"
6,"RandomForest_b, 0.8159","RandomForest_c, 0.4897","RandomForest_c, 0.1208","RandomForest_c, 0.5222","RandomForest_c, 0.2474","RandomForest_c, 24.7357"
7,"RandomForest_a, 0.8157","RandomForest_f, 0.4637","IRV_d, 0.0841","RandomForest_f, 0.4903","RandomForest_f, 0.2431","RandomForest_f, 24.3098"
8,"MultiClassification_b, 0.7532","IRV_e, 0.4356","SingleClassification_b, 0.0824","IRV_e, 0.4554","IRV_d, 0.2343","IRV_d, 23.431"
9,"IRV_e, 0.7463","IRV_d, 0.4324","IRV_e, 0.0752","IRV_d, 0.4459","IRV_b, 0.2123","IRV_b, 21.2277"


### Ranking by DTK+Means

In [5]:
display('Ranking by DTK+Means. Shows model-rank pairs.', complex_ranked_cv[req_metrics])

'Ranking by DTK+Means. Shows model-rank pairs.'

,ROC AUC RMI-FANCM FP,BEDROC AUC RMI-FANCM FP,PR auc.integral RMI-FANCM FP,NEF AUC RMI-FANCM FP,NEF_1 % RMI-FANCM FP,EF_1 % RMI-FANCM FP
0,"RandomForest_h, 1.0","RandomForest_h, 1.0","SingleClassification_b, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0","RandomForest_h, 1.0"
1,"RandomForest_g, 2.0","RandomForest_g, 2.0","SingleClassification_a, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0","RandomForest_g, 2.0"
2,"RandomForest_e, 3.0","RandomForest_e, 3.0","RandomForest_h, 2.0","RandomForest_e, 3.0","RandomForest_d, 3.0","RandomForest_d, 3.0"
3,"RandomForest_d, 4.0","RandomForest_d, 4.0","RandomForest_g, 4.0","RandomForest_b, 4.0","RandomForest_e, 4.0","RandomForest_e, 4.0"
4,"RandomForest_c, 5.0","RandomForest_b, 5.0","RandomForest_e, 5.0","RandomForest_d, 5.0","RandomForest_f, 5.0","RandomForest_f, 5.0"
5,"RandomForest_f, 6.0","RandomForest_c, 6.0","RandomForest_b, 6.0","RandomForest_a, 6.0","IRV_d, 6.0","IRV_d, 6.0"
6,"RandomForest_b, 6.0","RandomForest_a, 6.0","RandomForest_a, 7.0","RandomForest_c, 7.0","IRV_a, 7.0","IRV_a, 7.0"
7,"RandomForest_a, 8.0","RandomForest_f, 8.0","RandomForest_d, 8.0","RandomForest_f, 8.0","SingleClassification_b, 8.0","SingleClassification_b, 8.0"
8,"MultiClassification_b, 9.0","IRV_d, 9.0","RandomForest_c, 9.0","IRV_d, 9.0","SingleClassification_a, 9.0","SingleClassification_a, 9.0"
9,"IRV_e, 10.0","IRV_e, 10.0","MultiClassification_b, 9.0","MultiClassification_b, 10.0","RandomForest_c, 10.0","RandomForest_c, 10.0"


---
## Regret of best model

In [6]:
for m in req_metrics:
    simple_model = simple_top_n[m].iloc[0]
    simple_model = simple_model[:simple_model.index(',')]
    simple_top_n[m].iloc[0] = simple_model
    complex_model = complex_top_n[m].iloc[0]
    complex_model = complex_model[:complex_model.index(',')]
    complex_top_n[m].iloc[0] = complex_model

In [7]:
final_df = simple_top_n.T
final_df = pd.merge(final_df, complex_top_n.T, left_index=True, right_index=True)
final_df.columns = ['Best by Mean','Best by DTK+Mean']
final_df.columns.name = 'Metric'
final_df.index = [s.replace(' RMI-FANCM FP', '') for s in final_df.index]

display(final_df)

Metric,Best by Mean,Best by DTK+Mean
ROC AUC,RandomForest_h,RandomForest_h
BEDROC AUC,RandomForest_h,RandomForest_h
PR auc.integral,RandomForest_h,SingleClassification_b
NEF AUC,RandomForest_h,RandomForest_h
NEF_1 %,RandomForest_h,RandomForest_h
EF_1 %,RandomForest_h,RandomForest_h
